In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **Predict**

In [ ]:
!pip install gradio ultralytics pillow torch torchvision pillow-heif


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.6 MB/s eta 0:00:00


In [ ]:
import torch
from torchvision import models, transforms
from torchvision.models import MobileNet_V2_Weights
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import gradio as gr
from ultralytics import YOLO
from pillow_heif import register_heif_opener

register_heif_opener()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

yolo_weight_path = "/content/drive/MyDrive/nhandangmau/nhandangmau/weight/yolov8n_best.pt"
yolo_model = YOLO(yolo_weight_path).to(device)

mobilenet = models.mobilenet_v2(weights=None)
mobilenet.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.3),
    torch.nn.Linear(1280, 43)
)
mobilenet.load_state_dict(torch.load(
    "/content/drive/MyDrive/nhandangmau/nhandangmau/weight/mobilenetv2_test.pth",
    map_location=device
))
mobilenet.to(device)
mobilenet.eval()

weights = MobileNet_V2_Weights.IMAGENET1K_V1
tf_mobilenet = transforms.Compose([
    transforms.Resize((224, 224)),
    weights.transforms()
])

def detect_and_classify(image):
    img = Image.fromarray(image).convert("RGB")
    results = yolo_model(img)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    cls_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()

    # font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", size=32)

    for i, (box, cls_id) in enumerate(zip(boxes, cls_ids)):
        xmin, ymin, xmax, ymax = [int(b) for b in box]
        crop = img.crop((xmin, ymin, xmax, ymax))
        x = tf_mobilenet(crop).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = mobilenet(x)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            pred_cls = probs.argmax(dim=1).item()
            conf = probs[0, pred_cls].item()

        label = f"Class {pred_cls}"
        draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=3)
        draw.text((xmin, max(0, ymin - 12)), label, fill="yellow", font=font)

    return np.array(img)

demo = gr.Interface(
    fn=detect_and_classify,
    inputs=gr.Image(type="numpy", label="Tải ảnh lên"),
    outputs=gr.Image(label="Kết quả phát hiện & phân loại"),
    title="🚦 Nhận dạng biển báo giao thông (YOLO + MobileNetV2)",
    description="Tải ảnh lên để hệ thống phát hiện và phân loại biển báo giao thông."
)

demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a13e204a7e306b96e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",